In [1]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

In [2]:
non_sarc_df = pd.read_csv("../data/corpora_dataset.csv",sep=";", encoding='cp1252')

sarc_df = pd.read_csv("../data/corpora_dataset.csv", sep=";", encoding='cp1252')

In [3]:
non_sarc_df.shape, sarc_df.shape

((9789, 3), (5721, 3))

In [4]:
df = pd.concat([non_sarc_df, sarc_df], axis=0)

In [5]:
df.shape

(15510, 3)

In [6]:
df.head()

,titles,category,Label
0,"Roma, una perdita d’acqua in via del Forte Bra...",cronaca,0
1,"Bari, racket di prostituzione minorile: alle t...",cronaca,0
2,La truffa al figlio di Bruno Giordano: un fals...,cronaca,0
3,"Modena, la ragazzina stuprata a 12 anni e il v...",cronaca,0
4,Fedez denunciato per rissa in discoteca per il...,cronaca,0


In [7]:
sentences = df['titles'].values
labels = df['Label'].values

# Veriyi eğitim ve test olarak ayırın
training_sentences, testing_sentences, training_labels, testing_labels = train_test_split(sentences, labels, test_size=0.2)

vocab_size = 1000
embedding_dim = 16
max_length = 32
trunc_type='post'
padding_type='post'

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

model = Sequential([
    Embedding(vocab_size, embedding_dim),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(24, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

num_epochs = 10
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

history = model.fit(
    training_padded, training_labels,
    epochs=num_epochs,
    validation_data=(testing_padded, testing_labels),
    verbose=2
)

loss, accuracy = model.evaluate(testing_padded, testing_labels, verbose=2)
print(f'Test Accuracy: {accuracy}')

Epoch 1/10
388/388 - 2s - 6ms/step - accuracy: 0.7702 - loss: 0.4488 - val_accuracy: 0.8614 - val_loss: 0.3162
Epoch 2/10
388/388 - 1s - 2ms/step - accuracy: 0.8792 - loss: 0.2757 - val_accuracy: 0.8740 - val_loss: 0.2941
Epoch 3/10
388/388 - 1s - 2ms/step - accuracy: 0.9072 - loss: 0.2211 - val_accuracy: 0.8765 - val_loss: 0.2943
Epoch 4/10
388/388 - 1s - 2ms/step - accuracy: 0.9244 - loss: 0.1857 - val_accuracy: 0.8801 - val_loss: 0.2929
Epoch 5/10
388/388 - 1s - 2ms/step - accuracy: 0.9382 - loss: 0.1577 - val_accuracy: 0.8930 - val_loss: 0.2767
Epoch 6/10
388/388 - 1s - 2ms/step - accuracy: 0.9508 - loss: 0.1294 - val_accuracy: 0.8981 - val_loss: 0.2996
Epoch 7/10
388/388 - 1s - 2ms/step - accuracy: 0.9622 - loss: 0.1043 - val_accuracy: 0.9043 - val_loss: 0.2888
Epoch 8/10
388/388 - 1s - 2ms/step - accuracy: 0.9718 - loss: 0.0808 - val_accuracy: 0.9033 - val_loss: 0.3293
Epoch 9/10
388/388 - 1s - 2ms/step - accuracy: 0.9792 - loss: 0.0616 - val_accuracy: 0.9039 - val_loss: 0.3366
E